## Mesh Extraction
Run the following code to extract a mesh given a .pth file name. Change the pth_file variable to that of the model you would like to create a mesh for.

In [1]:
import mcubes
import trimesh
import torch
import numpy as np

In [2]:
device = 'cuda'
pth_file = 'nerf_models/monkey_biz.pth'
model = torch.load(pth_file).to(device)


In [3]:
N = 100
scale = 1.5

x = torch.linspace(-scale, scale, N)
y = torch.linspace(-scale, scale, N)
z = torch.linspace(-scale, scale, N)

x, y, z = torch.meshgrid((x, y, z))

xyz = torch.cat((x.reshape(-1, 1),
                 y.reshape(-1, 1),
                 z.reshape(-1, 1)), dim=1)

/home/eherrin@ad.ufl.edu/anaconda3/lib/python3.11/site-packages/torch/functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3549.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [4]:
with torch.no_grad():
    _, density = model.forward(xyz.to(device), torch.zeros_like(xyz).to(device))
    
density = density.cpu().numpy().reshape(N, N, N)

In [5]:
vertices, triangles = mcubes.marching_cubes(density, 5 * np.mean(density))
mesh = trimesh.Trimesh(vertices / N, triangles)
mesh.show()